In [12]:
import os
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import functools

# --- CARICAMENTO DATI E TOKENIZER ---

# Assicurati che il path a utils.py sia corretto
import sys
sys.path.insert(0, '..')
from utils import load_captions_txt, data_generator_single_example

# Carica le caption e crea il tokenizer
print("Caricamento caption e creazione tokenizer...")
captions_dict = load_captions_txt("../data/Flickr8k_text/captions.txt")
all_captions = [cap for caps_list in captions_dict.values() for cap in caps_list]

tokenizer = Tokenizer(oov_token="<unk>")
tokenizer.fit_on_texts(all_captions)

vocab_size = len(tokenizer.word_index) + 1
max_length = max(len(c.split()) for c in all_captions)

print(f"Vocabulary size: {vocab_size}")
print(f"Max caption length: {max_length}")

# Carica le feature delle immagini
print("Caricamento feature immagini...")
with open("features/image_features.pkl", "rb") as f:
    image_features = pickle.load(f)

print("\n✔️ Risorse caricate.")

Caricamento caption e creazione tokenizer...
Vocabulary size: 8497
Max caption length: 40
Caricamento feature immagini...

✔️ Risorse caricate.


In [13]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
import os

# --- CARICAMENTO DEL MODELLO PER CONTINUARE L'ADDESTRAMENTO ---

# Definisci i percorsi per i modelli
original_model_path = "models/model_caption.keras"
finetuned_model_path = "models/model_caption_finetuned.keras"

# Logica: se esiste un modello "finetuned", carica quello. Altrimenti, parti dall'originale.
if os.path.exists(finetuned_model_path):
    model_path_to_load = finetuned_model_path
    print(f"✔️ Trovato modello già rifinito. Caricamento da: {model_path_to_load}")
else:
    model_path_to_load = original_model_path
    print(f"⚠️ Nessun modello rifinito trovato. Partenza dal modello originale: {model_path_to_load}")

# Carica il modello selezionato
model = load_model(model_path_to_load)

# --- PREPARAZIONE PER IL FINE-TUNING ---

# Imposta un learning rate basso per il fine-tuning
new_learning_rate = 0.0001
print(f"Impostazione del learning rate: {new_learning_rate}")

# Ricompila il modello per applicare il nuovo learning rate
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=new_learning_rate),
    metrics=['accuracy']
)

print("\nModello pronto per il fine-tuning.")
model.summary()

✔️ Trovato modello già rifinito. Caricamento da: models/model_caption_finetuned.keras
Impostazione del learning rate: 0.0001

Modello pronto per il fine-tuning.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 40)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 40, 256)   │  2,175,232 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 40, 256)   │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, 40)        │          0 │ input_layer_1[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer         │ (None, 2048)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 40, 256)   │    525,312 │ dropout_1[0][0],  │
│                     │                   │            │ not_equal_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 2048)      │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 40, 256)   │          0 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    524,544 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 256)       │    525,312 │ dropout_2[0][0],  │
│                     │                   │            │ not_equal_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 256)       │          0 │ dense[0][0],      │
│                     │                   │            │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │     65,792 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 8497)      │  2,183,729 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,999,921 (22.89 MB)

 Trainable params: 5,999,921 (22.89 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
batch_size = 64

# Definizione della signature del dataset
output_signature_single = (
    (
        tf.TensorSpec(shape=(2048,), dtype=tf.float32),          
        tf.TensorSpec(shape=(max_length,), dtype=tf.int32)      
    ),
    tf.TensorSpec(shape=(vocab_size,), dtype=tf.float32)         
)

# Creazione del generatore parziale
# ... (codice identico a prima) ...

dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Calcolo degli step
steps_per_epoch = len(captions_dict) // batch_size
print(f"Dataset pronto. {steps_per_epoch} steps per epoca.")

Dataset pronto. 126 steps per epoca.


In [16]:
# --- ESECUZIONE DEL FINE-TUNING ITERATIVO ---

# Decidi per quante epoche addestrare IN QUESTA SESSIONE
epochs_this_session = 5 # Puoi cambiare questo valore ogni volta che esegui

print(f"\nInizio addestramento per altre {epochs_this_session} epoche...")

history = model.fit(
    dataset,
    epochs=epochs_this_session,
    steps_per_epoch=steps_per_epoch, 
    verbose=1
)

# --- SALVATAGGIO DEL MODELLO AGGIORNATO ---

print("\nAddestramento completato per questa sessione. Salvataggio del modello aggiornato...")

# Salviamo SEMPRE con il nome del modello rifinito.
# Questo garantisce che la prossima esecuzione caricherà questa versione migliorata.
# La cartella 'models' dovrebbe già esistere, ma ricontrolliamo.
os.makedirs("models", exist_ok=True)
model.save(finetuned_model_path)

print(f"✔️ Modello aggiornato e salvato correttamente in: {finetuned_model_path}")


Inizio addestramento per altre 5 epoche...
Epoch 1/5
126/126 ━━━━━━━━━━━━━━━━━━━━ 32s 254ms/step - accuracy: 0.3456 - loss: 3.4733
Epoch 2/5
126/126 ━━━━━━━━━━━━━━━━━━━━ 106s 842ms/step - accuracy: 0.3558 - loss: 3.4550
Epoch 3/5
126/126 ━━━━━━━━━━━━━━━━━━━━ 31s 244ms/step - accuracy: 0.3459 - loss: 3.4019
Epoch 4/5
126/126 ━━━━━━━━━━━━━━━━━━━━ 43s 341ms/step - accuracy: 0.3510 - loss: 3.4031
Epoch 5/5
126/126 ━━━━━━━━━━━━━━━━━━━━ 53s 425ms/step - accuracy: 0.3523 - loss: 3.4179

Addestramento completato per questa sessione. Salvataggio del modello aggiornato...
✔️ Modello aggiornato e salvato correttamente in: models/model_caption_finetuned.keras
